In [ ]:
import yaml
import os
import psycopg2
import redis
from dotenv import load_dotenv
import json
import hashlib
from datetime import datetime

In [ ]:
load_dotenv()

In [3]:
pg_user = os.getenv('POSTGRES_USER')
pg_password = os.getenv('POSTGRES_PASSWORD')
pg_db = os.getenv('POSTGRES_DB')
pg_host = os.getenv('POSTGRES_HOST', 'localhost')
pg_port = os.getenv('POSTGRES_PORT', '5432')

redis_host = os.getenv('REDIS_HOST', 'localhost')
redis_port = os.getenv('REDIS_PORT', '6379')
redis_password = os.getenv('REDIS_PASSWORD')

In [4]:
# Load database schema from YAML file
with open('../database_schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

In [5]:
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_db,
    user=pg_user,
    password=pg_password,
    port=pg_port
)

In [6]:
pg_cursor = pg_conn.cursor()
print("Successfully connected to PostgreSQL.")

Successfully connected to PostgreSQL.


In [7]:
redis_conn = redis.Redis(
    host=redis_host,
    port=int(redis_port),
    password=redis_password,
    decode_responses=True
)

redis_conn.ping()  # Check if connection is active
print("Successfully connected to Redis.")

Successfully connected to Redis.


In [8]:
def initialize_postgres():
    """Create all tables with USER_AUTH as the combined user table"""
    
    
    # Creat salt_email table
    print("Creating salt_email table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS salt_email (
        email VARCHAR(255) PRIMARY KEY,
        salt VARCHAR(255) NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    
    # Create extension for UUID generation
    print("Creating pgcrypto extension...")
    pg_cursor.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")
    
    # Create CHAIN_DETAILS table
    print("Creating CHAIN_DETAILS table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS CHAIN_DETAILS (
        chain_id VARCHAR(255) PRIMARY KEY,
        chain VARCHAR(100) NOT NULL,
        chain_status VARCHAR(50) NOT NULL,
        chain_explorer VARCHAR(255)
    );
    """)
    
    # Create USER_AUTH table (combined with USERS)
    print("Creating USER_AUTH table (combined with USERS)...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS USER_AUTH (
        user_pub_key VARCHAR(255) PRIMARY KEY,
        username VARCHAR(100),
        email VARCHAR(255),
        chain_id VARCHAR REFERENCES CHAIN_DETAILS(chain_id),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    
    # Create AGENT table
    print("Creating AGENT table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS AGENT (
        agent_id VARCHAR(255) PRIMARY KEY,
        creation_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        owner VARCHAR(255) REFERENCES USER_AUTH(user_pub_key),
        status VARCHAR(20) CHECK (status IN ('Active', 'Not Published')),
        tags JSONB,
        license VARCHAR(255),
        fork VARCHAR(255),
        socials JSONB,
        description TEXT,
        name VARCHAR(255) NOT NULL,
        chain_id VARCHAR REFERENCES CHAIN_DETAILS(chain_id)
    );
    """)
    
    # Create UNPUBLISHED_AGENT table
    print("Creating UNPUBLISHED_AGENT table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS UNPUBLISHED_AGENT (
        agent_id VARCHAR(255) PRIMARY KEY REFERENCES AGENT(agent_id),
        last_edited_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        workflow JSONB NOT NULL,
        md5 VARCHAR(32) NOT NULL
    );
    """)
    
    # Create PUBLISHED_AGENT table
    print("Creating PUBLISHED_AGENT table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS PUBLISHED_AGENT (
        agent_id VARCHAR(255) PRIMARY KEY REFERENCES AGENT(agent_id),
        last_edited_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        workflow JSONB NOT NULL,
        md5 VARCHAR(32) NOT NULL
    );
    """)
    
    # Create METADATA table
    print("Creating METADATA table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS METADATA (
        agent_id VARCHAR(255) PRIMARY KEY REFERENCES AGENT(agent_id),
        markdown_object JSONB NOT NULL
    );
    """)
    
    # Create ACCESS_LEVEL_TABLE table
    print("Creating ACCESS_LEVEL_TABLE table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS ACCESS_LEVEL_TABLE (
        access_level INTEGER PRIMARY KEY,
        access_level_name VARCHAR(50) NOT NULL,
        descriptions_and_permissions JSONB
    );
    """)
    
    # Create CONTRACT_DETAILS table
    print("Creating CONTRACT_DETAILS table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS CONTRACT_DETAILS (
        contract_id VARCHAR(255) PRIMARY KEY,
        chain_id VARCHAR REFERENCES CHAIN_DETAILS(chain_id),
        contract_name VARCHAR(100) NOT NULL,
        contract_version VARCHAR(20) NOT NULL
    );
    """)
    
    # Create BLOCKCHAIN_AGENT_DATA table
    print("Creating BLOCKCHAIN_AGENT_DATA table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS BLOCKCHAIN_AGENT_DATA (
        agent_id VARCHAR(255) PRIMARY KEY REFERENCES AGENT(agent_id),
        version VARCHAR(255) NOT NULL,
        published_date TIMESTAMP NOT NULL,
        published_hash VARCHAR(255) NOT NULL,
        contract_id VARCHAR(255) REFERENCES CONTRACT_DETAILS(contract_id),
        nft_id VARCHAR(255) NOT NULL,
        nft_mint_trx_id VARCHAR(255) NOT NULL
    );
    """)
    
    # Create NFT_ACCESS table
    print("Creating NFT_ACCESS table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS NFT_ACCESS (
        user_id VARCHAR(255) REFERENCES USER_AUTH(user_pub_key),
        nft_id VARCHAR(255) NOT NULL,
        access_level INTEGER REFERENCES ACCESS_LEVEL_TABLE(access_level),
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (user_id, nft_id)
    );
    """)
    
    # Create CONVERSATIONS table (updated to reference USER_AUTH)
    print("Creating CONVERSATIONS table (referencing USER_AUTH)...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS CONVERSATIONS (
        conversation_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
        user_id VARCHAR(255) REFERENCES USER_AUTH(user_pub_key),
        title VARCHAR(255),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    
    # Create MESSAGES table
    print("Creating MESSAGES table...")
    pg_cursor.execute("""
    CREATE TABLE IF NOT EXISTS MESSAGES (
        message_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
        conversation_id UUID REFERENCES CONVERSATIONS(conversation_id),
        sender_type VARCHAR(10) CHECK (sender_type IN ('user', 'assistant')),
        agent_id VARCHAR(255) REFERENCES AGENT(agent_id),
        content TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    
    # Commit the transaction
    pg_conn.commit()
    print("All tables created successfully!")

In [9]:
# print("Creating salt_email table...")
# pg_cursor.execute("""
# CREATE TABLE IF NOT EXISTS salt_email (
#     email VARCHAR(255) PRIMARY KEY,
#     salt VARCHAR(255) NOT NULL,
#     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# );

# """)
# pg_conn.commit()

In [10]:
# delete all tables

def delete_all_tables():
    """Delete all tables in the database."""
    print("Deleting all tables...")
    pg_cursor.execute("DROP TABLE IF EXISTS salt_email CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS CHAIN_DETAILS CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS USER_AUTH CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS AGENT CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS UNPUBLISHED_AGENT CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS PUBLISHED_AGENT CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS METADATA CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS ACCESS_LEVEL_TABLE CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS CONTRACT_DETAILS CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS BLOCKCHAIN_AGENT_DATA CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS NFT_ACCESS CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS CONVERSATIONS CASCADE;")
    pg_cursor.execute("DROP TABLE IF EXISTS MESSAGES CASCADE;")

    # Commit the transaction
    pg_conn.commit()
    print("All tables deleted successfully!")

In [11]:
# delete_all_tables()

In [12]:
def initialize_redis():
    # Get Redis key definitions from schema
    redis_keys = schema.get('redis', {}).get('keys', {})
    
    # Create sample template for each Redis key type
    for key_name, key_def in redis_keys.items():
        key_pattern = key_def.get('key_pattern', '')
        key_type = key_def.get('type', 'hash')
        ttl = key_def.get('ttl', 0)
        fields = key_def.get('fields', [])
        
        # Create documentation keys with examples to help developers
        doc_key = f"schema:{key_name}:doc"
        
        # Create an example key based on the pattern type
        example_key = ""
        if key_name == 'USER_SESSION':
            # For user session, use a JWT token example
            example_key = key_pattern.format(token='eyJhbGciOiJIUzI1NiI...')
        elif key_name == 'WORKFLOW_LIVE_EDITOR':
            # For workflow editor, use an agent_id example
            example_key = key_pattern.format(agent_id='agent456')
        else:
            # For any other key type, provide a generic example
            try:
                # Try to format with user_id if that placeholder exists
                example_key = key_pattern.format(user_id='user123')
            except KeyError:
                # If user_id placeholder doesn't exist, just use the pattern as is
                example_key = key_pattern
        
        # Create documentation with field examples
        doc_data = {
            'key_pattern': key_pattern,
            'type': key_type,
            'ttl': ttl,
            'fields': [{'name': field.get('name'), 'description': field.get('description')} for field in fields],
            'example': example_key
        }
        
        redis_conn.set(doc_key, json.dumps(doc_data))
        print(f"Created Redis schema documentation for {key_name}")
    
    # Create metadata about synchronization system
    sync_config = schema.get('synchronization', {})
    redis_conn.set('schema:sync:config', json.dumps(sync_config))
    print("Created Redis synchronization configuration")

In [13]:
initialize_postgres()


Creating salt_email table...
Creating pgcrypto extension...
Creating CHAIN_DETAILS table...
Creating USER_AUTH table (combined with USERS)...
Creating AGENT table...
Creating UNPUBLISHED_AGENT table...
Creating PUBLISHED_AGENT table...
Creating METADATA table...
Creating ACCESS_LEVEL_TABLE table...
Creating CONTRACT_DETAILS table...
Creating BLOCKCHAIN_AGENT_DATA table...
Creating NFT_ACCESS table...
Creating CONVERSATIONS table (referencing USER_AUTH)...
Creating MESSAGES table...
All tables created successfully!


In [14]:
initialize_redis()

Created Redis schema documentation for USER_SESSION
Created Redis schema documentation for WORKFLOW_LIVE_EDITOR
Created Redis synchronization configuration


In [15]:
# check the tables created

pg_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = pg_cursor.fetchall()
print("Tables created in PostgreSQL:")
for table in tables:
    print(table[0])

Tables created in PostgreSQL:
salt_email
chain_details
user_auth
agent
unpublished_agent
published_agent
metadata
contract_details
blockchain_agent_data
nft_access
access_level_table
conversations
messages
